<a href="https://colab.research.google.com/github/mm46655/mynewrepository/blob/master/Copy_of_Best_Available_Pixel_Composite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BEST AVAILABLE PIXEL COMPOSITE (BAP) *in Google Earth Engine Python API*
based on *Pixel-Based Image Compositing for Large-Area Dense
Time Series Applications and Science (White, 2014)*
https://goo.gl/Fi8fCY

In [1]:
!pip install earthengine-api==0.1.175 #earth-engine Python API

     |████████████████████████████████| 122kB 9.8MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 2.3MB 22.8MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.175-cp36-none-any.whl size=144180 sha256=744ab493f2cd66219c164569aae4e8ad880de7eca79b455393fecd1964a342f1
  Stored in directory: /root/.cache/pip/wheels/ee/b9/f9/7278e3dd64c0520f3658c9404ade74ab2a06b9b104e6de16f6
Successfully built earthengine-api
  Found existing installation: earthengine-api 0.1.209
    Uninstalling earthengine-api-0.1.209:
      Successfully uninstalled earthengine-api-0.1.209


In [2]:
!earthengine authenticate 

Opening the following address in a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

Please authorize access to your Earth Engine account, and paste the generated code below. If the web browser does not start, please manually browse the URL above.

Please enter authorization code: 4/vwH_ncGIDECcPJ4I21PDyVQkKRG8MoXZkk37SaxIWhtItEYEZdMFpiU

Successfully saved authorization token.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# Earth Engine Python API
import ee
ee.Initialize()

In [8]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.8.3


In [0]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [6]:
pip install geebap

     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 225kB 26.9MB/s 
  Created wheel for geebap: filename=geebap-0.2.5-cp36-none-any.whl size=50433 sha256=eaa9b244fc9e11a599b216fd15a0311da98f4dd9529b7be32e7b33683426b969
  Stored in directory: /root/.cache/pip/wheels/60/ab/25/8865b36541224338cbc0e7f6e19cbf437b5d84aacfd572083d
  Created wheel for simpleeval: filename=simpleeval-0.9.10-cp36-none-any.whl size=13758 sha256=5b32d35fe2c7191ead5dc369faef8f9726eab2d51694c524f62b61013aa6e825
  Stored in directory: /root/.cache/pip/wheels/20/ab/91/eeb7746161f88e6ea897483494ab6034ebce9884b8f2dea7e5
  Created wheel for geetools: filename=geetools-0.4.13-cp36-none-any.whl size=96146 sha256=e3b93d7b84433ed7ae50240d8e203eb7bc1b8d9c3ac142c55c992ec7f17fd4dd
  Stored in directory: /root/.cache/pip/wheels/02/89/a3/1cceb1525740f73dd5caa9e09cb8821f1113881ca84832fee6
  Created wheel for pyshp: filename=pyshp-2.1.0-

### Map of Any Image within Study Site

In [10]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
  .filterDate('2019-05-01', '2019-09-30')\
  .filterBounds(ee.Geometry.Polygon(-76.68783507813652,35.27929957693104,\
  -75.64413390626152,35.27929957693104,\
  -75.64413390626152,36.02026592377204,\
  -76.68783507813652,36.02026592377204,\
  -76.68783507813652,35.27929957693104))

# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', )
image.getInfo()

first=collection.first()
first.getInfo()

dicc = {
    'Least Cloudy' : image.getMapId({'bands': ['B4', 'B3', 'B2'],'min':0,'max':2000}),
    'First Image': first.getMapId({'bands': ['B4','B3','B2'],'min':0,'max':2000})
}

# Display the results
center = [ 35.69,-75.22]
Mapdisplay(center, dicc, zoom_start= 9)


Least cloudy image: 


In [88]:
import geebap
from geetools import tools

import pprint
pp = pprint.PrettyPrinter(indent=2)

# SEASON
a_season = geebap.Season('05-15', '09-15')

# MASKS
cld_mask = geebap.masks.Mask()

# Combine masks in a tuple
masks = (cld_mask,)

# FILTERS
filt_cld = geebap.filters.CloudCover()
# filt_mask = geebap.filters.MaskCover() # Doesn't work

# Combine filters in a tuple
filters = (filt_cld,)#, filt_mask)

# SCORES
best_doy = geebap.scores.Doy('06-15', a_season)
sat = geebap.scores.Satellite()
out = geebap.scores.Outliers(("ndvi",))
ind = geebap.scores.Index("ndvi")
maskpercent = geebap.scores.MaskPercentKernel()
dist = geebap.scores.CloudDist()

# Combine scores in a tuple
scores = (
    best_doy,
    sat,
    out,
    ind,
    maskpercent,
    dist
)

# BAP OBJECT
BAP = geebap.Bap(range=(0, 0),
                 season=a_season,
                 masks=masks,
                 scores=scores,
                 filters=filters)

# SITE
site = ee.Geometry.Polygon(-76.68783507813652,35.27929957693104,\
  -75.64413390626152,35.27929957693104,\
  -75.64413390626152,36.02026592377204,\
  -76.68783507813652,36.02026592377204,\
  -76.68783507813652,35.27929957693104)
#site = ee.Geometry.Polygon(-76.42881419914642,35.30148730801318,
#   -76.04017284172454,35.30148730801318,
#   -76.04017284172454,35.459362004001804,
#   -76.42881419914642,35.459362004001804,
#   -76.42881419914642,35.30148730801318)

# COMPOSITE
composite = BAP.build_composite_best(2019, site=site, indices=("ndvi",))

composite.getInfo()


{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'red'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'swir2'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'nir'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'blue'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'thermal'},
  {'crs': 'EPSG:4326',
 

In [102]:
#print(site.getInfo()['coordinates'])
mid = site.centroid()
center1 = mid.getInfo()['coordinates']
center = reversed(center1)
print(list(center))

[35.38055447188363, -76.23449352043582]


In [104]:
bands = {
    'Image' : composite.getMapId({'bands': ['ndvi']})
}

#bands = composite.getMapId({
#    'bands': ['red','green','blue'], 
#    'min': 0, 
#    'max': 4000})
Mapdisplay(center,bands,zoom_start=9)


In [0]:
pip install geetools


In [0]:
from geetools import batch

In [0]:
sAPP = ee.Geometry.Polygon(-76.42881419914642,35.30148730801318,
   -76.04017284172454,35.30148730801318,
   -76.04017284172454,35.459362004001804,
   -76.42881419914642,35.459362004001804,
   -76.42881419914642,35.30148730801318)
pppAPP = ee.Geometry.Polygon(-76.31338582346655,35.87036027390846,
    -76.00164876291967,35.87036027390846,
    -76.00164876291967,36.00378840591169,
    -76.31338582346655,36.00378840591169,
    -76.31338582346655,35.87036027390846)

fc = ee.FeatureCollection([sAPP,pppAPP])

In [0]:
batch.image.toDriveByFeature(composite, collection=fc, property='site', folder='BAP_fc', name='APP_BAP', scale=30, dataType='float')